In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## Step 1: ਸੰਰਚਿਤ ਆਉਟਪੁੱਟ ਲਈ ਪਾਇਡੈਂਟਿਕ ਮਾਡਲ ਪਰਿਭਾਸ਼ਤ ਕਰੋ

ਇਹ ਮਾਡਲ ਐਜੰਟਾਂ ਵੱਲੋਂ ਵਾਪਸ ਕੀਤੇ ਜਾਣ ਵਾਲੇ **ਸਕੀਮਾ** ਨੂੰ ਪਰਿਭਾਸ਼ਤ ਕਰਦੇ ਹਨ। ਪਾਇਡੈਂਟਿਕ ਨਾਲ `response_format` ਵਰਤਣ ਨਾਲ ਇਹ ਯਕੀਨੀ ਬਣਦਾ ਹੈ:
- ✅ ਪ੍ਰਕਾਰ-ਸੁਰੱਖਿਅਤ ਡੇਟਾ ਨਿੱਕਾਸ਼
- ✅ ਸਵੈਚਾਲਿਤ ਮਾਨਤਾ
- ✅ ਮੁਫਤ-ਪਾਠੀ ਜਵਾਬਾਂ ਤੋਂ ਕੋਈ ਵਿਸ਼ਲੇਸ਼ਣ ਤ੍ਰੁੱਟੀਆਂ ਨਹੀਂ
- ✅ ਖੇਤਰਾਂ ਦੇ ਆਧਾਰ 'ਤੇ ਆਸਾਨ ਸ਼ਰਤੀ ਰੂਟਿੰਗ


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## ਕਦਮ 2: ਹੋਟਲ ਬੁਕਿੰਗ ਟੂਲ ਬਣਾਓ

ਇਹ ਟੂਲ ਉਹ ਹੈ ਜੋ **availability_agent** ਕਾਲ ਕਰੇਗਾ ਇਹ ਜਾਣਚ ਕਰਨ ਲਈ ਕਿ ਕੀ ਕਮਰੇ ਉਪਲਬਧ ਹਨ। ਅਸੀਂ `@ai_function` ਡੈਕੋਰੇਟਰ ਦਾ ਵਰਤੋਂ ਕਰਦੇ ਹਾਂ:
- ਇੱਕ Python ਫੰਕਸ਼ਨ ਨੂੰ AI-ਕਾਲ ਕਰਨ ਵਾਲੇ ਟੂਲ ਵਿੱਚ ਬਦਲਣ ਲਈ
- LLM ਲਈ JSON schema ਆਟੋਮੈਟਿਕ ਤੌਰ 'ਤੇ ਬਣਾਉਣ ਲਈ
- ਪੈਰਾਮੀਟਰ ਦੀ ਵੈਧਤਾ ਸੰਭਾਲਣ ਲਈ
- ਏਜੰਟਾਂ ਵਲੋਂ ਆਟੋਮੈਟਿਕ ਕਾਲਿੰਗ ਨੂੰ ਯੋਗ ਬਣਾਉਣ ਲਈ

ਇਸ ਡੈਮੋ ਲਈ:
- **ਸਟੌਕਹੋਮ, ਸੀਅਟਲ, ਟੋਕਿਓ, ਲੰਡਨ, ਐਮਸਟਰਡੈਂਮ** → ਕਮਰੇ ਉਪਲਬਧ ਹਨ ✅
- **ਸਾਰੇ ਹੋਰ ਸ਼ਹਿਰ** → ਕੋਈ ਕਮਰੇ ਨਹੀਂ ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Step 3: ਰਾਊਟਿੰਗ ਲਈ ਸ਼ਰਤ ਫੰਕਸ਼ਨ ਤੈਅ ਕਰੋ

ਇਹ ਫੰਕਸ਼ਨ ਏਜੰਟ ਦੇ ਜਵਾਬ ਦੀ ਜਾਂਚ ਕਰਦੇ ਹਨ ਅਤੇ ਵਰਕਫਲੋ ਵਿੱਚ ਕਿਹੜਾ ਰਾਸ਼ਤਾ ਲੈਣਾ ਹੈ, ਇਹ ਨਿਰਧਾਰਤ ਕਰਦੇ ਹਨ।

**ਮੁੱਖ ਪੈਟਰਨ:**
1. ਚੈੱਕ ਕਰੋ ਕਿ ਸੁਨੇਹਾ `AgentExecutorResponse` ਹੈ
2. ਢਾਂਚাগত ਨਿਕਾਸ (Pydantic ਮਾਡਲ) ਨੂੰ ਪਾਰਸ ਕਰੋ
3. ਰਾਊਟਿੰਗ ਨੂੰ ਨਿਯੰਤਰਿਤ ਕਰਨ ਲਈ `True` ਜਾਂ `False` ਵਾਪਸ ਕਰੋ

ਵਰਕਫਲੋ ਇਹ ਸ਼ਰਤਾਂ **ਐੱਜਾਂ** 'ਤੇ ਮੁਲਾਂਕਣ ਕਰੇਗਾ ਤਾਂ ਜੋ ਨਿਰਧਾਰਤ ਕਰ ਸਕੇ ਕਿ ਅਗਲਾ ਐਗਜ਼ੀਕਿਊਟਰ ਕੌਣ ਹੈ।


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## ਕਦਮ 4: ਕਸਟਮ ਡਿਸਪਲੇ ਇਜ਼ੈਕਿਊਟਰ ਬਣਾਓ

ਇਜ਼ੈਕਿਊਟਰ ਉਹ ਵਰਕਫਲੋ ਕੰਪੋਨੇਟ ਹਨ ਜੋ ਰੂਪਾਂਤਰਣ ਜਾਂ ਸਾਈਡ ਐਫੈਕਟ ਕਰਦੇ ਹਨ। ਅਸੀਂ ਅੰਤਿਮ ਨਤੀਜੇ ਨੂੰ ਦਿਖਾਉਣ ਲਈ `@executor` ਡੇਕਰੇਟਰ ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਇੱਕ ਕਸਟਮ ਇਜ਼ੈਕਿਊਟਰ ਬਣਾਉਂਦੇ ਹਾਂ।

**ਮੁੱਖ ਧਾਰਨਾਵਾਂ:**
- `@executor(id="...")` - ਇੱਕ ਫੰਕਸ਼ਨ ਨੂੰ ਵਰਕਫਲੋ ਇਜ਼ੈਕਿਊਟਰ ਵਜੋਂ ਰਜਿਸਟਰ ਕਰਦਾ ਹੈ
- `WorkflowContext[Never, str]` - ਇਨਪੁਟ/ਆਉਟਪੁੱਟ ਲਈ ਟਾਈਪ ਹਿੰਟ
- `ctx.yield_output(...)` - ਅੰਤਿਮ ਵਰਕਫਲੋ ਨਤੀਜਾ ਪ੍ਰਦਾਨ ਕਰਦਾ ਹੈ


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## Step 5: ਵਾਤਾਵਰਣ ਚਲਕਾਂ ਲੋਡ ਕਰੋ

LLM ਕਲਾਇੰਟ ਨੂੰ ਸੰਰਚਿਤ ਕਰੋ। ਇਹ ਉਦਾਹਰਨ ਇਨ੍ਹਾਂ ਨਾਲ ਕੰਮ ਕਰਦੀ ਹੈ:
- **GitHub ਮਾਡਲਜ਼** (GitHub ਟੋਕਨ ਨਾਲ ਮੁਫ਼ਤ ਟੀਅਰ)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## ਕਦਮ 6: ਸੰਰਚਿਤ ਆਊਟਪੁੱਟ ਨਾਲ AI ਏਜੰਟ ਬਣਾਉਣਾ

ਅਸੀਂ **ਤਿੰਨ ਵਿਸ਼ੇਸ਼ ਏਜੰਟ ਬਣਾਉਂਦੇ ਹਾਂ**, ਜੋ ਹਰ ਇੱਕ `AgentExecutor` ਵਿੱਚ ਲਪੇਟੇ ਜਾਂਦੇ ਹਨ:

1. **availability_agent** - ਟੂਲ ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਹੋਟਲ ਦੀ ਉਪਲਬਧਤਾ ਜਾਂਚਦਾ ਹੈ
2. **alternative_agent** - ਬਦਲਾਵ ਲਈ বিকল্প ਸ਼ਹਿਰ ਸੁਝਾਉਂਦਾ ਹੈ (ਜਦੋਂ ਕੋਈ ਕਮਰੇ ਨਾ ਹੋਣ)
3. **booking_agent** - ਬੁਕਿੰਗ ਨੂੰ ਪ੍ਰਫੁੱਲਿਤ ਕਰਦਾ ਹੈ (ਜਦੋਂ ਕਮਰੇ ਉਪਲਬਧ ਹੋਣ)

**ਮੁੱਖ ਵਿਸ਼ੇਸ਼ਤਾਵਾਂ:**
- `tools=[hotel_booking]` - ਏਜੰਟ ਨੂੰ ਟੂਲ ਪ੍ਰਦਾਨ ਕਰਦਾ ਹੈ
- `response_format=PydanticModel` - ਸੰਰਚਿਤ JSON ਆਊਟਪੁੱਟ ਲਈ ਮਜ਼ਬੂਰ ਕਰਦਾ ਹੈ
- `AgentExecutor(..., id="...")` - ਵਰਕਫਲੋ ਲਈ ਏਜੰਟ ਨੂੰ ਲਪੇਟਦਾ ਹੈ


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## Step 7: ਸਥਿਤੀਪੂਰਣ ਏਜਾਂ ਨਾਲ ਵਰਕਫਲੋ ਬਣਾਉਣਾ

ਹੁਣ ਅਸੀਂ `WorkflowBuilder` ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਸਥਿਤੀਪੂਰਣ ਰੂਟਿੰਗ ਨਾਲ ਗ੍ਰਾਫ਼ ਤਿਆਰ ਕਰਦੇ ਹਾਂ:

**ਵਰਕਫਲੋ ਸੰਰਚਨਾ:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**ਮੁੱਖ ਢੰਗ:**
- `.set_start_executor(...)` - ਐਂਟਰੀ ਪੁਆਇੰਟ ਸੈੱਟ ਕਰਦਾ ਹੈ
- `.add_edge(from, to, condition=...)` - ਸਥਿਤੀਪੂਰਣ ਏਜ ਜੋੜਦਾ ਹੈ
- `.build()` - ਵਰਕਫਲੋ ਨੂੰ ਅੰਤਮ ਰੂਪ ਦਿੰਦਾ ਹੈ


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## Step 8: ਟੈਸਟ ਕੇਸ 1 ਚਲਾਓ - ਸ਼ਹਿਰ ਬਿਨਾਂ ਉਪਲਬਧਤਾ (ਪੈਰਿਸ)

ਆਓ **ਕੋਈ ਉਪਲਬਧਤਾ ਨਾ ਹੋਣ** ਵਾਲਾ ਰਸਤਾ ਟੈਸਟ ਕਰੀਏ ਜਿਸ ਵਿੱਚ ਅਸੀਂ ਪੈਰਿਸ ਵਿੱਚ ਹੋਟਲ ਮੰਗਦੇ ਹਾਂ (ਜਿੱਥੇ ਸਾਡੇ ਸਿਮੂਲੇਸ਼ਨ ਵਿੱਚ ਕੋਈ ਕਮਰੇ ਨਹੀਂ ਹਨ)।


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## Step 9: ਦੌੜਾਓ ਟੈਸਟ ਕੇਸ 2 - ਸ਼ਹਿਰ ਨਾਲ ਮੌਜੂਦਗੀ (ਸਟਾਕਹੋਲਮ)

ਹੁਣ ਆਓ **ਮੌਜੂਦਗੀ** ਮਾਰਗ ਦੀ ਜਾਂਚ ਕਰੀਏ ਸਟਾਕਹੋਲਮ ਵਿੱਚ ਹੋਟਲਾਂ ਦੀ ਬੇਨਤੀ ਕਰਕੇ (ਜਿਸ ਵਿੱਚ ਸਾਡੇ ਗਣਕ ਵਿੱਚ ਕਮਰੇ ਹਨ)।


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## ਮੁੱਖ ਗੱਲਾਂ ਅਤੇ ਅਗਲੇ ਕਦਮ

### ✅ ਤੁਸੀਂ ਕੀ ਸਿਖਿਆ ਹੈ:

1. **WorkflowBuilder ਪੈਟਰਨ**
   - ਐਨਟਰੀ ਪੋਇੰਟ ਨੂੰ ਪਰਿਭਾਸ਼ਿਤ ਕਰਨ ਲਈ `.set_start_executor()` ਦੀ ਵਰਤੋਂ ਕਰੋ
   - ਸਸ਼ਰਤ ਰੂਟਿੰਗ ਲਈ `.add_edge(from, to, condition=...)` ਦੀ ਵਰਤੋਂ ਕਰੋ
   - ਵਰਕਫਲੋ ਨੂੰ ਫਾਇਨਲ ਕਰਨ ਲਈ `.build()` ਕਾਲ ਕਰੋ

2. **ਸਸ਼ਰਤ ਰੂਟਿੰਗ**
   - ਸਸ਼ਰਤ ਫੰਕਸ਼ਨ `AgentExecutorResponse` ਦੀ ਜਾਂਚ ਕਰਦੇ ਹਨ
   - ਰੂਟਿੰਗ ਫੈਸਲੇ ਕਰਨ ਲਈ ਸੰਰਚਿਤ ਆਉਟਪੁਟ ਨੂੰ ਪਾਰਸ ਕਰੋ
   - ਇੱਕ ਏਜ਼ ਨੂੰ ਸਰਗਰਮ ਕਰਨ ਲਈ `True` ਵਾਪਸ ਕਰੋ, ਛੱਡਣ ਲਈ `False`

3. **ਟੂਲ ਇੰਟੀਗ੍ਰੇਸ਼ਨ**
   - Python ਫੰਕਸ਼ਨਾਂ ਨੂੰ AI ਟੂਲਜ਼ ਵਿੱਚ ਬਦਲਣ ਲਈ `@ai_function` ਦੀ ਵਰਤੋਂ ਕਰੋ
   - ਜਰੂਰਤ ਪੈਂਦੇ ਹੀ ਏਜੰਟ ਟੂਲ ਆਟੋਮੈਟਿਕਲੀ ਕਾਲ ਕਰਦੇ ਹਨ
   - ਟੂਲ JSON ਵਾਪਸ ਕਰਦੇ ਹਨ ਜਿਸਨੂੰ ਏਜੰਟ ਪਾਰਸ ਕਰ ਸਕਦੇ ਹਨ

4. **ਸੰਰਚਿਤ ਆਉਟਪੁਟ**
   - ਟਾਈਪ-ਸੇਫ਼ ਡਾਟਾ ਨਿਕਾਸ ਲਈ Pydantic ਮਾਡਲ ਦੀ ਵਰਤੋਂ ਕਰੋ
   - ਏਜੰਟ ਬਣਾਉਂਦੇ ਸਮੇਂ `response_format=MyModel` ਸੈੱਟ ਕਰੋ
   - ਜਵਾਬਾਂ ਨੂੰ `Model.model_validate_json()` ਨਾਲ ਪਾਰਸ ਕਰੋ

5. **ਕਸਟਮ ਏਗਜ਼ਿਕਿਊਟਰ**
   - ਵਰਕਫਲੋ ਕੰਪੋਨੇਟ ਬਣਾਉਣ ਲਈ `@executor(id="...")` ਦੀ ਵਰਤੋਂ ਕਰੋ
   - ਏਗਜ਼ਿਕਿਊਟਰ ਡਾਟਾ ਨੂੰ ਬਦਲ ਸਕਦੇ ਹਨ ਜਾਂ ਸਾਈਡ ਇਫੈਕਟ ਕਰ ਸਕਦੇ ਹਨ
   - ਵਰਕਫਲੋ ਨਤੀਜੇ ਦੇਣ ਲਈ `ctx.yield_output()` ਦੀ ਵਰਤੋਂ ਕਰੋ

### 🚀 ਅਸਲ ਦੁਨੀਆ ਦੇ ਐਪਲੀਕੇਸ਼ਨ:

- **ਟ੍ਰੈਵਲ ਬੁਕਿੰਗ**: ਉਪਲਬਧਤਾ ਚੈੱਕ ਕਰੋ, ਵਿਵਿਕਲਪ ਸਿਫਾਰਸ਼ ਕਰੋ, ਚੋਣਾਂ ਦੀ ਤੁਲਨਾ ਕਰੋ
- **ਕਸਟਮਰ ਸਰਵਿਸ**: ਮਸਲੇ ਦੇ ਕਿਸਮ, ਭਾਵਨਾ, ਤਰਜੀਹ ਦੇ ਆਧਾਰ ਤੇ ਰੂਟ ਕਰੋ
- **ਈ-ਕਾਮਰਸ**: ਇਨਵੈਂਟਰੀ ਚੈੱਕ ਕਰੋ, ਵਿਵਿਕਲਪ ਦਿਓ, ਆਰਡਰ ਪ੍ਰੋਸੈਸ ਕਰੋ
- **ਸੰਪਾਦਕੀ ਨਿਗਰਾਨੀ**: ਵੈਸ਼ਮਿਆ ਸਕੋਰ, ਯੂਜ਼ਰ ਫਲੈਗ ਦੇ ਆਧਾਰ ਤੇ ਰੂਟ ਕਰੋ
- **ਮਨਜ਼ੂਰੀ ਵਰਕਫਲੋ**: ਰਕਮ, ਯੂਜ਼ਰ ਰੋਲ, ਖਤਰਾ ਪੱਧਰ ਦੇ ਆਧਾਰ ਤੇ ਰੂਟ ਕਰੋ
- **ਬਹੁ-ਚਰਣ ਪ੍ਰਕਿਰਿਆ**: ਡਾਟਾ ਗੁਣਵੱਤਾ, ਪੂਰਨਤਾ ਦੇ ਆਧਾਰ ਤੇ ਰੂਟ ਕਰੋ

### 📚 ਅਗਲੇ ਕਦਮ:

- ਹੋਰ ਜਟਿਲ ਸਸ਼ਰਤਾਂ ਸ਼ਾਮਿਲ ਕਰੋ (ਕਈ ਮਿਯਾਰ)
- ਵਰਕਫਲੋ ਸਟੇਟ ਮੈਨੇਜਮੈਂਟ ਨਾਲ ਲੂਪ ਲਾਗੂ ਕਰੋ
- ਦੁਬਾਰਾ ਵਰਤੇ ਜਾਣ ਵਾਲੇ ਕੰਪੋਨੇਟ ਲਈ ਸਬ-ਵਰਕਫਲੋ ਸ਼ਾਮਿਲ ਕਰੋ
- ਅਸਲੀ APIs (ਹੋਟਲ ਬੁਕਿੰਗ, ਇਨਵੈਂਟਰੀ ਸਿਸਟਮ) ਨਾਲ ਇੰਟੀਗ੍ਰੇਸ਼ਨ ਕਰੋ
- ਗਲਤੀ ਸੰਭਾਲ ਅਤੇ ਬੈਕਅਪ ਰਸਤੇ ਸ਼ਾਮਿਲ ਕਰੋ
- ਬਿਲਟ-ਇਨ ਵਿਜ਼ੂਅਲਾਈਜ਼ੇਸ਼ਨ ਟੂਲਜ਼ ਨਾਲ ਵਰਕਫਲੋਜ਼ ਦਾ ਵਿਜ਼ੂਅਲ ਕਰੋ


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**ਡਿਸਕਲੇਮਰ**:  
ਇਹ ਦਸਤਾਵੇਜ਼ AI ਅਨੁਵਾਦ ਸੇਵਾ [Co-op Translator](https://github.com/Azure/co-op-translator) ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਅਨੁਵਾਦ ਕੀਤਾ ਗਿਆ ਹੈ। ਜਦੋਂ ਕਿ ਅਸੀਂ ਸਹੀਤਾ ਲਈ ਕੋਸ਼ਸ਼ ਕਰਦੇ ਹਾਂ, ਕਿਰਪਾ ਕਰਕੇ ਧਿਆਨ ਦੇਵੋ ਕਿ ਆਟੋਮੈਟਿਕ ਅਨੁਵਾਦਾਂ ਵਿੱਚ ਗਲਤੀਆਂ ਜਾਂ ਅਸਸਥਿਰਤਾਵਾਂ ਹੋ ਸਕਦੀਆਂ ਹਨ। ਆਪਣੀ ਮੂਲ ਭਾਸ਼ਾ ਵਿੱਚ ਮੂਲ ਦਸਤਾਵੇਜ਼ ਨੂੰ ਅਧਿਕਾਰਤ ਸਰੋਤ ਮੰਨਿਆ ਜਾਣਾ ਚਾਹੀਦਾ ਹੈ। ਮਹੱਤਵਪੂਰਣ ਜਾਣਕਾਰੀ ਲਈ ਪੇਸ਼ੇਵਰ ਮਨੁੱਖੀ ਅਨੁਵਾਦ ਦੀ ਸਿਫਾਰਸ਼ ਕੀਤੀ ਜਾਂਦੀ ਹੈ। ਇਸ ਅਨੁਵਾਦ ਦੀ ਵਰਤੋਂ ਤੋਂ ਹੋਣ ਵਾਲੀਆਂ ਕਿਸੇ ਵੀ ਗਲਤਫਹਿਮੀਆਂ ਜਾਂ ਭੁੱਲਾਂ ਲਈ ਅਸੀਂ ਜ਼ਿੰਮੇਵਾਰ ਨਹੀਂ ਹਾਂ।
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
